# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [15]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [16]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [17]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [18]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a,"\n")
print(a,"\n")
print(L@u)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [19]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [20]:
np.linalg.matrix_rank(a1)

6

In [21]:
l, u = diy_lu(a1)
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


<ipython-input-15-8918e3eddeb9>:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-15-8918e3eddeb9>:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-15-8918e3eddeb9>:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-15-8918e3eddeb9>:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [22]:
# ... ENTER YOUR CODE HERE ...
def LUDecompositionCandidate(mat):
    """
    check wether a matrix can be represented as a LU factorization
    by trying to factorize and check if at any point the leading minor is 0
    """
    N = mat.shape[0]
    
    u = mat.copy()
    for j in range(N-1):
        lam = np.eye(N)
        if(u[j,j] == 0.0): return False
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
    return True

In [23]:
print("Can be LU factored?",LUDecompositionCandidate(a))
print("Can be LU factored?",LUDecompositionCandidate(a1))

Can be LU factored? True
Can be LU factored? False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [24]:
def diy_lu(a):
    """Construct the PLU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    
    I used the following tutorial to understand and implement this algorithm:
    https://www.nagwa.com/en/explainers/976193728703/
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        row = j
        for rows in range(j+1,N):
            if abs(u[rows,j]) > abs(u[row,j]):
                row = rows
        u[[row,j]],u[[j,row]] = u[[j,row]],u[[row,j]]
        L[:,row],L[:,j] = L[:,j],L[:,row].copy()
        P[[row,j]],P[[j,row]] = P[[j,row]],P[[row,j]]
        L[[row,j]],L[[j,row]] = L[[j,row]],L[[row,j]]
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
        
        lam[j+1:, j] = gamma
        L = L @ lam
    return P,L,u
        

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [25]:
P,L,U = diy_lu(a1)
print("checking PA = LU")
print("this is PA\n",P @ a1,"\n")
print("this is LU\n",L @ U,"\n")
print("checking A = P⁻1LU")
print("this is A\n",a1)
P = np.transpose(P)
print("this is P⁻1LU\n",P @ L @ U)

checking PA = LU
this is PA
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    0.75  0.429 0.3   0.231 0.188]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]] 

this is LU
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    0.75  0.429 0.3   0.231 0.188]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]] 

checking A = P⁻1LU
this is A
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
this is P⁻1LU
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.

### Results

Even though the steps taking to make the PLU matrix factorization correct, we get a rounder error